<a href="https://colab.research.google.com/github/YousefAtefB/StanceCat-COV19/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/StanceCat-COV19

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/StanceCat-COV19


In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import nltk
from tqdm import tqdm
from sklearn.metrics import f1_score
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data PreProcessing

In [11]:
train = pd.read_csv('Dataset/train.csv')
text_train, category_train, stance_train = train['text'], train['category'], train['stance']

dev = pd.read_csv('Dataset/dev.csv')
text_dev, category_dev, stance_dev = dev['text'], dev['category'], dev['stance']

In [12]:
train.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [13]:
dev.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_ك...,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كور...,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأول...,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف م...,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة...",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونش...,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF>...,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقي...,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! ...,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا م...,info_news,1


In [14]:
text_train, category_train, stance_train = np.array(train['text']), np.array(train['category']), np.array(train['stance'])
text_dev, category_dev, stance_dev = np.array(dev['text']), np.array(dev['category']), np.array(dev['stance'])

print(text_train.shape, category_train.shape, stance_train.shape)
print(text_dev.shape, category_dev.shape, stance_dev.shape)

(6988,) (6988,) (6988,)
(1000,) (1000,) (1000,)


In [15]:
!pip install arabert
!pip install transformers
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name="aubmindlab/bert-base-arabertv02-twitter"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)
  
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 179 kB 30.3 MB/s 
     |████████████████████████████████| 184 kB 73.5 MB/s 
     |████████████████████████████████| 126 kB 67.3 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=8c4b41ba85ceda6aa73f604322eaf3790bfdc581fec94739ba6454de2f77d401
  Stored in directory: /root/.cache/pip/wheels/71/4d/3c/cada364d4ea0026deee7208dee1e61bcebd20aa2ae5dc154ba
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 23.8 MB/s 
     |████████████████████████████████| 7.6 MB 60.4 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 


Downloading:   0%|          | 0.00/476 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/751k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/667 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/541M [00:00<?, ?B/s]

In [16]:
example_text = text_train[0]
bert_input = tokenizer(example_text, padding='max_length', max_length = 100, truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

output = model(**bert_input)
print(output.logits.shape)

tensor([[    2,  6941, 28652, 16136, 34495,    10, 17448,   196, 13593,   306,
           650,  5166, 48342,   197,   139,   391,  5787,  7325,   197,   139,
           391, 11474,   139,  7829,   213,  3616,   185, 54749,   305,  2858,
         10639,   139,  1721,   338,  2779, 17613,  1686,   298,  5263,     1,
           634,   889, 13591,  5175,   323, 28892,    20,    20,    20,  9110,
           394,   418,  3682, 28892,  1422,   418,  6843, 36720,   306,  5263,
         62279,    77,   221,   221,   178,   223,    31,     1,     1,    89,
            20,    72,   219,     1,    54,   275,   272,   244,   244,   270,
           260,   264,   268,   179,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
categories = set(category_train)
print(categories)
category2id = {word:i for i, word in enumerate(list(categories))}
print(category2id['celebrity'])

{'advice', 'requests', 'unrelated', 'restrictions', 'personal', 'info_news', 'plan', 'rumors', 'celebrity', 'others'}
8


# Model Building

## BERT

### Ideas to try
1) bi-directional
2) pre-training
3) multi-layers
4) BERT
5) transformers notebook
6) packed_padded_sequences
7) pre-trained embedding

### Building Model

In [18]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y):

    x = x.copy()

    self.X = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in x]

    self.Y = torch.tensor(y)

    self.len = len(x)

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [19]:
stance_train_dataset = Dataset(text_train, stance_train + 1)
category_train_dataset = Dataset(text_train, [category2id[category] for category in category_train])

stance_dev_dataset = Dataset(text_dev, stance_dev + 1)
category_dev_dataset = Dataset(text_dev, [category2id[category] for category in category_dev])

In [115]:
from transformers import BertModel
class BertClassifier(nn.Module):

    def __init__(self, output_dim, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('aubmindlab/bert-base-arabertv02-twitter')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, output_dim)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [116]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Training

In [129]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train, val, learning_rate, epochs):

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                print(train_input['input_ids'].shape)

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                print(output.shape)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    
                    print(val_input['input_ids'].shape)

                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)


                    output = model(input_id, mask)

                    print(output.shape)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train): .3f} \
                | Train Accuracy: {total_acc_train / len(train): .3f} \
                | Val Loss: {total_loss_val / len(val): .3f} \
                | Val Accuracy: {total_acc_val / len(val): .3f}')

In [ ]:
EPOCHS = 5
model = BertClassifier(3)
LR = 1e-6
              
train(model, stance_train_dataset, stance_dev_dataset, LR, EPOCHS)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

torch.Size([2, 1, 512])
torch.Size([2, 3])


  0%|          | 1/3494 [00:00<14:33,  4.00it/s]

torch.Size([2, 1, 512])
torch.Size([2, 3])


  0%|          | 2/3494 [00:00<13:40,  4.26it/s]

...

 11%|█         | 389/3494 [01:27<11:31,  4.49it/s]

torch.Size([2, 1, 512])
torch.Size([2, 3])


 11%|█         | 390/3494 [01:27<11:30,  4.49it/s]

torch.Size([2, 1, 512])
torch.Size([2, 3])


 11%|█         | 391/3494 [01:27<11:28,  4.51it/s]

In [125]:
torch.save(model.state_dict(), 'stance.pt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/StanceCat-COV19


In [ ]:
def train(model, train_dataset, train_dataloader, optimizer, criterion, clip):
    
    model.train()
        
    total_acc_train = 0.0
    total_loss_train = 0.0

    for train_input, train_label, src_len in tqdm(train_dataloader):

        train_input = train_input.to(device).permute(1, 0)
        # print(train_input.shape)
        train_label = train_label.to(device)

        output = model(train_input, src_len)
        # print(output.shape, train_label.shape)
        
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        train_label = train_label.view(-1)

        batch_loss = criterion(output, train_label)

        total_loss_train += batch_loss
        
        acc = torch.sum(torch.argmax(output, -1) == train_label) 
        total_acc_train += acc

        optimizer.zero_grad()

        batch_loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()

    # calculate loss    
    epoch_loss = total_loss_train / len(train_dataset)
    
    # calculate accuracy
    epoch_acc = total_acc_train / len(train_dataset)

    # calculate f1 score
    train_input, train_label, src_len = train_dataset[:]
    train_input = train_input.to(device).permute(1, 0)
    train_label = train_label.to(device)
    output = model(train_input, src_len)

    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    train_label = train_label.view(-1)
    y_true, y_pred = train_label, torch.argmax(output, -1)

    f1_macro = f1_score(y_true, y_pred, average='macro')

    return epoch_loss, epoch_acc, f1_macro

In [ ]:
def evaluate(model, test_dataset, test_dataloader, criterion):

  model.eval()

  total_acc_test = 0.0
  total_loss_test = 0.0
  
  with torch.no_grad():

    for test_input, test_label, src_len in tqdm(test_dataloader):

      test_input = test_input.to(device).permute(1, 0)
      test_label = test_label.to(device)


      output = model(test_input, src_len)

      batch_loss = criterion(output.view(-1, model.output_dim), test_label.view(-1))

      total_loss_test += batch_loss

      acc = torch.sum(torch.argmax(output, -1)==test_label)
      total_acc_test += acc

    # calculate loss
    total_loss_test /= len(test_dataset)

    # calculate accuracy
    total_acc_test /= len(test_dataset)

    # calculate f1 score
    test_input, test_label, src_len = test_dataset[:]
    test_input = test_input.to(device).permute(1, 0)
    test_label = test_label.to(device)
    output = model(test_input, src_len)

    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    test_label = test_label.view(-1)
    y_true, y_pred = test_label, torch.argmax(output, -1)

    f1_macro = f1_score(y_true, y_pred, average='macro')
  
  return total_loss_test, total_acc_test, f1_macro

In [ ]:
def train_evaluate(model, train_dataset, dev_dataset, model_name, batch_size=512, epochs=10, learning_rate=0.01, clip=1):

  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

  dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size)

  # criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
  criterion = nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  
  model = model.to(device)
  criterion = criterion.to(device)

  best_f1_macro = 0

  for epoch_num in range(epochs):

    epoch_loss, epoch_acc, train_f1_macro = train(model, train_dataset, train_dataloader, optimizer, criterion, clip)
    dev_loss, dev_acc, dev_f1_macro = evaluate(model, dev_dataset, dev_dataloader, criterion)

    if dev_f1_macro > best_f1_macro:
      best_f1_macro = dev_f1_macro
      torch.save(model.state_dict(), 'best_'+model_name+'.pt')

    print(f'Train = Epochs: {epoch_num + 1} | Loss: {epoch_loss} | Accuracy: {epoch_acc} | f1_macro : {train_f1_macro}')
    print(f'Dev = Epochs: {epoch_num + 1} | Loss: {dev_loss} | Accuracy: {dev_acc} | f1_macro : {dev_f1_macro}')    

  model.load_state_dict(torch.load('best_'+model_name+'.pt'))

  dev_loss, dev_acc, dev_f1_macro = evaluate(model, dev_dataset, dev_dataloader, criterion)

  print(f'Best Dev = Loss: {dev_loss} | Accuracy: {dev_acc} | f1_macro : {dev_f1_macro}')

In [ ]:
INPUT_DIM = vocab_size
OUTPUT_DIM = 3
EMB_DIM = 50 #256
HID_DIM = 50 #512
DROPOUT = 0.5

stance_model = LSTM(INPUT_DIM, EMB_DIM, HID_DIM, OUTPUT_DIM, DROPOUT).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
stance_model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(stance_model):,} trainable parameters')

SyntaxError: invalid decimal literal (4236606597.py, line 4)

In [ ]:
train_evaluate(stance_model, stance_train_dataset, stance_dev_dataset, 'stance_model')

In [ ]:
INPUT_DIM = vocab_size
OUTPUT_DIM = 10
EMB_DIM = 50 #256
HID_DIM = 50 #512
DROPOUT = 0.5

category_model = LSTM(INPUT_DIM, EMB_DIM, HID_DIM, OUTPUT_DIM, DROPOUT).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
category_model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(category_model):,} trainable parameters')

In [ ]:
train_evaluate(category_model, category_train_dataset, category_dev_dataset, 'category_model')

## Dumped

In [ ]:
evaluate(stance_model, stance_dev_dataset)

In [ ]:
evaluate(category_model, category_dev_dataset)

In [ ]:
def train(model, train_dataset, batch_size=512, epochs=10, learning_rate=0.01):
  
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

  # criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
  criterion = nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  
  model = model.to(device)
  criterion = criterion.to(device)

  for epoch_num in range(epochs):
    total_acc_train = 0.0
    total_loss_train = 0.0

    for train_input, train_label, src_len in tqdm(train_dataloader):

      train_input = train_input.to(device).permute(1, 0)
      # print(train_input.shape)
      train_label = train_label.to(device)

      output = model(train_input, src_len)

      # print(output.shape, train_label.shape)
      
      batch_loss = criterion(output.view(-1, model.output_dim), train_label.view(-1))

      total_loss_train += batch_loss
      
      acc = torch.sum(torch.argmax(output, -1) == train_label) 
      total_acc_train += acc

      optimizer.zero_grad()

      batch_loss.backward()

      optimizer.step()
      
    epoch_loss = total_loss_train / len(train_dataset)

    epoch_acc = total_acc_train / len(train_dataset)

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')


In [ ]:
def evaluate(model, test_dataset, batch_size=512):

  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  model = model.to(device)

  total_acc_test = 0.0
  
  with torch.no_grad():

    for test_input, test_label, src_len in tqdm(test_dataloader):

      test_input = test_input.to(device).permute(1, 0)
      test_label = test_label.to(device)


      output = model(test_input, src_len)

      acc = torch.sum(torch.argmax(output, -1)==test_label)
      total_acc_test += acc
    
    total_acc_test /= len(test_dataset)
  
  print(f'\nDev Accuracy: {total_acc_test}')